# Contando N-Gramas

In [5]:
from nltk.corpus import gutenberg
sents = list(gutenberg.sents('austen-emma.txt'))
sents[0]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']']

Primero vemos cómo imprimir todos los trigramas de una sola oración:

In [15]:
sent = sents[0]

n = 3  # trigramas
for i in range(len(sent) - n + 1):
    print(sent[i:i+n])

['[', 'Emma', 'by']
['Emma', 'by', 'Jane']
['by', 'Jane', 'Austen']
['Jane', 'Austen', '1816']
['Austen', '1816', ']']


Ahora veamos cómo contar los trigramas de todas las oraciones:

In [16]:
from collections import defaultdict

count = defaultdict(int)

for sent in sents:
    for i in range(len(sent) - n + 1):
        ngram = tuple(sent[i:i+n])  # los diccionarios no pueden guardar listas, pero sí tuplas
        count[ngram] += 1

In [17]:
count

defaultdict(int,
            {('profession', ',', 'domestic'): 1,
             ('could', 'I', 'help'): 1,
             ('how', 'very', 'like'): 1,
             ('in', 'resolving', 'to'): 1,
             ('--"', 'it', 'is'): 1,
             ('Some', 'people', 'were'): 1,
             (',', 'and', 'looks'): 1,
             ('him', 'in', 'person'): 1,
             ('to', 'be', 'made'): 5,
             ('giving', 'her', 'independence'): 1,
             ('she', 'was', 'threatened'): 1,
             ('--', 'from', 'her'): 1,
             ('very', 'respectfully', 'at'): 1,
             ('did', 'him', 'a'): 1,
             ('s', 'very', 'promising'): 1,
             ('not', 'be', 'quiet'): 1,
             ('home', ',', 'without'): 1,
             ('have', 'a', 'good'): 2,
             ('obliged', 'to', 'own'): 1,
             ('make', 'it', 'out'): 2,
             ('could', 'ride', 'to'): 1,
             ('to', 'impose', 'any'): 1,
             ('agreeable', ';', 'the'): 1,
             ('it',

El código sirve para n-gramas en general.

Tareas pendientes:
- Agregar marcadores de principio y final de oración
- Contar n-gramas y (n-1)-gramas al mismo tiempo.

# Generando Lenguaje Natural

El siguiente modelo de bigramas se aprende a partir de dos oraciones:
- "el gato come pescado"
- "la gata come salmón"


In [42]:
probs = {
    '<s>': {'el': 0.5, 'la': 0.5},
    #'<s>': {'el': 0.6, 'la': 0.2, 'los': 0.1, 'las': 0.1},
    'el': {'gato': 1.0},
    'gato': {'come': 1.0},
    'come': {'pescado': 0.5, 'salmón': 0.5},
    'pescado': {'.': 1.0},
    '.': {'</s>': 1.0},
    'la': {'gata': 1.0},
    'gata': {'come': 1.0},
    'salmón': {'.': 1.0},
}

list(probs['<s>'].items())  # convertir un diccionario a lista de pares

[('el', 0.5), ('la', 0.5)]

Cada entrada del diccionario contiene una distribución discreta finita para la palabra siguiente dada la palabra anterior. Samplear de una distribución discreta finita es tan fácil como samplear un número al azar entre 0 y 1 y ver en qué región cae (ver [Wikipedia](https://en.wikipedia.org/wiki/Pseudo-random_number_sampling#Finite_discrete_distributions)).

Empezamos sampleando la primer palabra:

In [37]:
from random import random

def sample(problist):
    r = random()  # entre 0 y 1
    i = 0
    word, prob = problist[0]
    acum = prob
    while r > acum:
        i += 1
        word, prob = problist[i]
        acum += prob
    
    return word

sample(list(probs['<s>'].items()))

'el'

Podemos ver que el resultado del sampleo se corresponde con las probabilidades:

In [38]:
results = [sample(list(probs['<s>'].items())) for i in range(1000)]

from collections import Counter
print(Counter(results))

Counter({'el': 598, 'la': 194, 'los': 113, 'las': 95})


**Observaciones:**
- Si se ordena la lista de probabilidades de mayor a menor, el sampling es más rápido.
- El sampling también se puede hacer usando [random.choices](https://docs.python.org/3/library/random.html#random.choices) de python
ó [random.choice](https://stackoverflow.com/questions/11373192/generating-discrete-random-variables-with-specified-weights-using-scipy-or-numpy) de numpy.

Ahora veamos cómo samplear una oración completa:

In [55]:
word = '<s>'
while word != '</s>':
    problist = list(probs[word].items())
    word = sample(problist)
    print(word)

la
gata
come
pescado
.
</s>


Acá se ve que se pueden generar oraciones nuevas (no vistas en tiempo de entrenamiento).

Tareas pendientes:
- adaptar el código a n-gramas en general: usar tuplas como claves en probs!
- precalcular las listas ordenadas de mayor a menor (ver sorted_prob en los tests)